### CS2101 - Programming for Science and Finance
Prof. Götz Pfeiffer<br />
School of Mathematical and Statistical Sciences<br />
University of Galway

***

# Week 8: Vectorized Operations; Convolutions

* `numpy` provides efficient **storage** for homogeneous multidimensional data.
* `numpy` also provides efficient **vectorized operations** on such data.
* **Convolution** is one such operation on arrays which has many applications in image processing.

In [ ]:
import numpy as np
from PIL import Image

## Vectorized Operations.

* Numpy arrays can be added, multiplied, ..., provided they have the **same shape**.
* Numpy's implementation of these **vectorized operations** is usually faster than native Python loops.

In [ ]:
a = np.arange(6).reshape(2, 3)
a

In [ ]:
rng = np.random.default_rng()
b = rng.random((2, 3))
b

* The sum `a + b` of arrays `a` and `b` is the array `c` whose entry in row $i$ and column $j$ is the sum of the entries in row $i$ and column $j$ of the arrays `a` and `b`:  $c_{ij} = a_{ij} + b_{ij}$.

In [ ]:
a + b

* The product `a * b` of arrays `a` and `b` is the array `c` whose entry in row $i$ and column $j$ is the product of the entries in row $i$ and column $j$ of the arrays `a` and `b`:  $c_{ij} = a_{ij} \times  b_{ij}$.

In [ ]:
a * b

* Note that this (coordinate-wise) multipliction is different from the matrix multiplication in Linear Algebra.
* But this concept is easy to apply to all kinds of operations…

In [ ]:
np.sin(b)

## Broadcasting.

* If the arrays `a` and `b` involved in such an operation do not have the same shape, numpy tries to "blow up" the smaller array to match the size of the larger array by a process called **broadcasting**.
* In the special case where `a` is a **scalar**, i.e., an array of shape `()`, it is treated like an array of shape `b.shape` with all its entries equal to `a`.

In [ ]:
a = np.array(5)
print(a)
a.shape

In [ ]:
a + b

In [ ]:
a * b

In [ ]:
b[1]

In [ ]:
b[1] = a
b

*  This **philosophy** of vectorized behavior might be useful for some of this week's homework...

## Convolutions.

* Convolution is an operator that takes two functions and creates a new function.
* For a more thorough discussion, see the [Wikipedia page](https://en.wikipedia.org/wiki/Convolution).

### The Continuous Case

* The perhaps most common situation is the continuous case, when we have two functions $f,g:\mathbb{R} \rightarrow \mathbb{R}$; their convolution $f \ast g$ is then defined by:
$$
(f \ast g)(t) = \int_{-\infty}^{\infty} f(\tau)g(t-\tau)\, d\tau.
$$
* For this to be defined there are some requirements on $f$ and $g$ (otherwise the integral might be infinite).
* We will not be dealing with the continuous case in this module.
* It might not be completely obvious from the definition, but the convolution operator is in fact **commutative**, i.e. we have
$$
f \ast g = g \ast f,
$$
this can be seen by changing variables in the integration.

## The Discrete Case.

* The integral can be thought of as a generalisation of a sum, so it should not come as a surprise that we can define a discrete version of the convolution.
* We will start by looking at the 1-dimensional case.

### 1D Convolution.

* Now we let $f$ and $g$ be functions $f,g : \mathbb{Z} \rightarrow \mathbb{R}$. Their convolution $f\star g : \mathbb{Z} \rightarrow \mathbb{R}$ is then defined by
$$
(f \ast g)(n) = \sum_{i = -\infty}^{+\infty} f(i)g(n-i).
$$
* This convolution is also commutative.
* There is functionality in NumPy for computing these 1D convolutions via the function `np.convolve(a, b)` where `a` and `b` are both 1D arrays.

#### Example: a noisy sin function

* As an example we'll look at the $\sin$ function, but with an added noise to it.
* The noise will be coming from the `gauss` function from the `random` module, which generates normally distributed numbers with a given mean and standard deviation.

In [ ]:
import math
import matplotlib.pyplot as plt

* We can use list comprehension to create lists of function values.

In [ ]:
xxx = range(50)
yyy = np.array([math.sin(2*math.pi*i/50) for i in xxx])
plt.plot(xxx, yyy)

* Or we use numpy's vectorized operations:

In [ ]:
xxx = np.arange(50)
yyy = np.sin(2*math.pi*xxx/50)
plt.plot(xxx, yyy)

* Generate some noise from a normal (Gaussian) distribution with  standard deviation $0.4$.

In [ ]:
rng = np.random.default_rng()

In [ ]:
noise = rng.normal(scale=0.4, size=xxx.shape)
plt.scatter(xxx, noise)

* Quick check: is this bell shaped?

In [ ]:
plt.hist(noise, bins=7)

In [ ]:
data = yyy + noise
plt.plot(xxx, data)

* We see that the graph of resulting curve is rather jagged.
* We can now take a rolling average of the original value and its ten closest neighbours to attempt to smoothen the curve.

In [ ]:
w = 11
kernel = np.ones(w)/w
kernel

In [ ]:
(data[:w] * kernel).sum()

* Pad data with 5 zeros on either side.

In [ ]:
data2 = np.zeros(60)
data2[5:-5] = data

In [ ]:
data2

In [ ]:
conv = [(data2[i:w+i] * kernel).sum() for i in range(50)]
plt.plot(xxx, conv)

* Luckily, numpy's `convolve` command does all of this automatically.
* Here, the parameter `'same'` that the convolved result should have the same length as the original `data`.

In [ ]:
plt.plot(xxx, np.convolve(data, kernel, 'same'))

* Instead of a straight average, we can let closer points be weighted higher than points further away.
* Let `v` be an array of values that decay exponentially with the distance from the middle element at position 5:
  $$
  v_i = e^{-\frac{(i-5)^2}{16}}
  $$

In [ ]:
v = np.array([math.exp(-((i-5)**2/16)) for i in range(11)])
v

In [ ]:
v = np.exp(-(np.arange(11)-5)**2/16)
v

* The sum of the entries in the kernel should be 1.

In [ ]:
kernel1 = v/v.sum()
kernel1

* We now get an even smoother curve.

In [ ]:
plt.plot(range(50), np.convolve(data, kernel1, 'same'))

* Let's plot all the curves together.
* The blue is the original, the orange is the plain average and the green is with the Gaussian kernel.

In [ ]:
plt.plot(yyy)     # blue
plt.plot(data)    # orange
plt.plot(np.convolve(data, kernel, 'same'))  # green
plt.plot(np.convolve(data, kernel1, 'same')) # red 

## 2D convolution

* Next up we'll look at convolutions of functions of two variables.
* The use case for us is pixels in images, so we will only deal with the discrete case.
* Here we let $f$ and $g$ be functions $f,g : \mathbb{Z}^2 \rightarrow \mathbb{R}$, and define their convolution to be
  $$
  (f\ast g)(m,n) = \sum_{i=-\infty}^{+\infty} \sum_{j=-\infty}^{+\infty} f(i,j)g(m-i,n-j)
  $$
* Unfortunately, there is no built-in functionality for 2D convolutions in NumPy, so we'll have to implement it ourselves.

#### An example image

* Let's generate an image of size 100 by 100 pixels, with two circles drawn on it.
* The low resolution is intentional, because it allows us to better see the blurring effect of applying a convolution.

In [ ]:
expic = np.zeros((100,100,3),dtype='uint8')
for i in range(expic.shape[0]):
    for j in range(expic.shape[1]):
        if (i-20)**2 + (j-15)**2 <= 10**2:
            # a red circle with centre (20,15) and radius 10
            expic[i,j,0]=255
        if (i-60)**2 + (j-55)**2 <= 25**2:
            # a blue circle with centre (60,55) and radius 25
            expic[i,j,2] = 255

Image.fromarray(expic)

* Using matplotlib, we can zoom in a bit.

In [ ]:
plt.imshow(expic[20:50,20:50])

* We can see that the image is rather blocky, so we might want to smoothen the circles by blurring.
* To this end, we will convolve the image with a $3 \times 3$ matrix in such a way that the intensity of a pixel will be the average of the  original intensity in the pixel and that of its neighbouring pixels.

In [ ]:
kernel = np.ones((3,3))/9
kernel

In [ ]:
avblur = np.zeros((100,100,3), dtype='uint8')
for r in range(1,99):
    for c in range(1,99):
        val = np.array([0,0,0])
        for i in range(3):
            for j in range(3):
                val = val + kernel[i, j]*expic[r-i+1, c-j+1]
        avblur[r,c,:] = np.uint8(val)



In [ ]:
plt.imshow(avblur)

* Let's zoom in here as well in the same way as on our original.

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(avblur[20:50,20:50])

#### Blurring with a Gaussian kernel

* We saw earlier that we might get even better results by having a Gaussian kernel for the convolution, so let's create one now.

* The entries in the matrix should decay exponentially with the distance from the middle entry at position (1,1):
  $$
  v_{ij} = e^{-\frac{(i-1)^2 + (j-1)^2}{2}}
  $$
* As before, the sum of the weights should be 1.

In [ ]:
v = np.array([[math.exp(-((i-1)**2+(j-1)**2)/2) for j in range(3)] for i in range(3)])
kernel1 = v/v.sum()
kernel1

* We can now compute the blurring.

In [ ]:
gaussblur = np.zeros((100,100,3), dtype='uint8')
for r in range(1,99):
    for c in range(1,99):
        val = np.array([0,0,0])
        for i in range(3):
            for j in range(3):
                val = val + kernel1[i, j]*expic[r-i+1, c-j+1]
        gaussblur[r,c,:] = np.uint8(val)

       

In [ ]:
plt.imshow(gaussblur)

* And zoom in again.
* The difference is not huge, but it's clearly there.

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(gaussblur[20:50,20:50])

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1, 2, 1)
plt.imshow(avblur[20:50,20:50])
plt.subplot(1,2,2)
plt.imshow(gaussblur[20:50,20:50])

## References

* Convolution [[wikipedia]](https://en.wikipedia.org/wiki/Convolution)

##  Exercises

* Using numpy's vectorized operations, find a more efficient way to convolute the image array `expic` with whatever kernel.